In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed
from lxml import etree

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)


params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [3]:
def changeparams():
    headers_pc = {
        'User-Agent': ua.random,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
#     try:
#         res = requests.get('https://www.health.com/diseases-conditions', headers=headers_pc, proxies= randomproxy)
#         # 将cookieJar数据转化为字典，s
#         cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
#     except Exception as e:
#         time.sleep(3)
#         headers_pc = {
#         'User-Agent': ua.random,
#         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
#         }
#         randomproxy = random.choice(proxieslist)
#         res = requests.get('https://www.health.com/diseases-conditions', headers=headers_pc, proxies= randomproxy)
#         # 将cookieJar数据转化为字典，s
#         cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy

In [4]:
def get_response(url, headers_pc, randomproxy):
    try:
        requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接
        
        r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, timeout=10)
        r.encoding = 'utf-8'
        return r.text
    except Exception as e:
        time.sleep(5)
        print(e)

    return None

In [5]:
def get_article_url(url, term, headers_pc, randomproxy):
    results = []
    try:

        html = get_response(url, headers_pc, randomproxy)
#             encoding = cchardet.detect(r.content)['encoding']
#             html = r.content.decode(encoding)
        if os.path.exists(f"D:/Intern_Project/6_health.com/术语详情页/{term}.html"):
            pbar.update(1)
            return None
        
        with open(f'D:/Intern_Project/6_health.com/术语详情页/{term}.html', 'w+', encoding='utf-8') as f:
             f.write(html)

#         selector = Selector(text=html)
#         articleUrl_list = selector.xpath(
#             "//div[@class='card__content']")
#         #print(len(termUrl_list))
#         for articleUrl in articleUrl_list:
#             url = articleUrl.xpath('./../@href').extract()[0]
#             title = articleUrl.xpath('./span/span/text()').extract()[0].strip()
#             # print(term)
#             tmp = {
#                 'title': title,
#                 'url': url
#             }
#             results.append(tmp)
        
        time.sleep(2)
        
        pbar.update(1)
    except Exception as e:
        print(e)
        time.sleep(2)

    return results

In [14]:
def task_monitor(task_list, mt_pool, results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
                results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [15]:
with open('D:/Intern_Project/6_health.com/术语链接列表.json', 'r', encoding='utf-8') as f:
        keyword_results = json.load(f)

all_craw_tasks = []
results = []

headers_pc, randomproxy = changeparams()

pbar = tqdm(total=len(keyword_results))

with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for keyword in keyword_results:
        headers_pc, randomproxy = changeparams()
        url = keyword['url']
        term = keyword['term']
        tmp_task = mt_pool.submit(get_article_url, url, term, headers_pc, randomproxy)
        all_craw_tasks.append(tmp_task)

    # 监测下载任务进展
    task_monitor(all_craw_tasks, mt_pool, results)

    with open('D:/Intern_Project/6_health.com/文章链接列表.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(results, indent=2, ensure_ascii=False))

  1%|▋                                                                                 | 1/112 [00:05<09:49,  5.31s/it]

HTTPSConnectionPool(host='www.health.com', port=443): Max retries exceeded with url: /condition/adhd?ie=UTF-8&wd=Python (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))HTTPSConnectionPool(host='www.health.com', port=443): Max retries exceeded with url: /condition/acne?ie=UTF-8&wd=Python (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

HTTPSConnectionPool(host='www.health.com', port=443): Max retries exceeded with url: /condition/anxiety?ie=UTF-8&wd=Python (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
HTTPSConnectionPool(host='www.health.com', port=443): Max retries exceeded with url: /condition/allergy?ie=UTF-8&wd=Python (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
HTTPSConnectionPool(host='www.health.com', port=443): Max retries exceeded with url: /condition/addiction?ie=UTF-8&wd=Python (Caused by SSLError(S

  9%|███████▏                                                                         | 10/112 [00:05<01:14,  1.37it/s]

HTTPSConnectionPool(host='www.health.com', port=443): Max retries exceeded with url: /condition/asthma?ie=UTF-8&wd=Python (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))


 17%|█████████████▋                                                                   | 19/112 [00:10<00:52,  1.76it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 111/112 [00:36<00:00,  3.15it/s]

HTTPSConnectionPool(host='www.health.com', port=443): Max retries exceeded with url: /condition/osteoarthritis?ie=UTF-8&wd=Python (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002546F7D55B0>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。')))


100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [00:40<00:00,  1.00it/s]

HTTPSConnectionPool(host='www.health.com', port=443): Max retries exceeded with url: /condition/uti?ie=UTF-8&wd=Python (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002546FA12C10>: Failed to establish a new connection: [WinError 10061] 由于目标计算机积极拒绝，无法连接。')))


TypeError: 'NoneType' object is not iterable

In [6]:
def findAllFile(base):
    for root, ds, fs in os.walk(base):
        for f in fs:
            fullname = os.path.join(root, f)
            yield fullname

In [7]:

results = []
for html_file in findAllFile("D:/Intern_Project/6_health.com/术语详情页"):
    html_content = open(html_file, 'r', encoding='utf-8').read()
    selector = Selector(text=html_content)
    articleUrl_list = selector.xpath(
        "//div[@class='card__content']")
    #print(len(termUrl_list))
    for articleUrl in articleUrl_list:
        url = articleUrl.xpath('./../@href').extract()[0]
        title = articleUrl.xpath('./span/span/text()').extract()[0].strip()
        # print(term)
        tmp = {
            'title': title,
            'url': url
        }
        results.append(tmp)
    
with open('D:/Intern_Project/6_health.com/文章链接列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(results, indent=2, ensure_ascii=False))

In [8]:
def get_response2(url, headers_pc):
    try:
        requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接
        
        r = requests.get(url, headers=headers_pc, params=params, timeout=10)
        r.encoding = 'utf-8'
        return r.text
    except Exception as e:
        time.sleep(5)
        print(e)

    return None

In [12]:
# 用来测试哪些页面能简单获取到


# headers_pc, randomproxy = changeparams()

# url = f'https://www.health.com/condition/acne'
# # r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
# try:
#     html = get_response2(url, headers_pc)
#     with open(f'D:/Intern_Project/6_health.com/Acne.html', 'w+', encoding='utf-8') as f:
#         f.write(html)
# except Exception as e:
#     print(e)


html_content = open('D:/Intern_Project/6_health.com/术语详情页/Acne.html', 'r', encoding='utf-8').read()

# tree = etree.HTML(html_content)
selector = Selector(text=html_content)
articleUrl_list = selector.xpath(
    "//div[@class='card__content']")
#print(len(termUrl_list))
for articleUrl in articleUrl_list:
    url = articleUrl.xpath('./../@href').extract()[0]
    title = articleUrl.xpath('./span/span/text()').extract()[0].strip()
    # print(term)
    tmp = {
        'title': title,
        'url': url
    }   
print(tmp)

{'title': "This Beauty Influencer Got Real About Zits—and Why She's Okay With Them", 'url': 'https://www.health.com/condition/acne/beauty-influencer-ingrid-nilsen-acne'}
